# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 28.2 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 6.6 MB 1.4 MB/s 
     |████████████████████████████████| 596 kB 52.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# загружаем модель
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

Дообучать GPT будем на одной из глав книги Джеймса Баркли "Эльфы. Во власти тьмы". 

In [ ]:
from google.colab import drive

ROOT_DIR = '/content/drive' 
drive.mount(ROOT_DIR)

# обучающие данные
train_path = 'drive/MyDrive/HSE_compling/HW_10/orcs_and_dwarfs.txt'

Mounted at /content/drive


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# создаем датасет
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)
  
# специальный класс, который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
# параметры обучения
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
      output_dir='drive/MyDrive/HSE_compling/HW_10/finetuned',
      overwrite_output_dir=True, 
      num_train_epochs=100, 
      per_device_train_batch_size=32, 
      per_device_eval_batch_size=32,  
      gradient_accumulation_steps=16
    )

trainer = Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=train_dataset,
      optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)
    )

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 99
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.6930136871337891, metrics={'train_runtime': 165.0563, 'train_samples_per_second': 59.98, 'train_steps_per_second': 0.606, 'total_flos': 323348889600000.0, 'train_loss': 0.6930136871337891, 'epoch': 100.0})

Попробуем сгенерировать несколько текстов разными методами.

In [ ]:
text_1 = '- Темнеет, - заметил эльф.'
text_2 = 'Вдалеке послышался голос.'
input_ids_1 = tokenizer.encode(text_1, return_tensors='pt').to(DEVICE)
input_ids_2 = tokenizer.encode(text_2, return_tensors='pt').to(DEVICE)

**Argmax**

In [ ]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1, 
                        do_sample=False,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=False,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
- Темнеет, - заметил эльф. - А может, нам стоит поискать что-нибудь еще? 

 - А что, есть что-нибудь? - полюбопытствовала Рена. 

 - Да, - кивнул эльф. - Есть

Sample 2
Вдалеке послышался голос. 

 — Что это? — нетерпеливо спросила Рена. 

 — Это, — ответил за нее Железный Жрец, — это волшебное кольцо, которое можно использовать в качестве талисмана.


In [ ]:
# если запустить генерацию еще раз, результат будет такой же
with torch.no_grad():
    out_1 = model.generate(input_ids_1, 
                        do_sample=False,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=False,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
- Темнеет, - заметил эльф. - А может, нам стоит поискать что-нибудь еще? 

 - А что, есть что-нибудь? - полюбопытствовала Рена. 

 - Да, - кивнул эльф. - Есть

Sample 2
Вдалеке послышался голос. 

 — Что это? — нетерпеливо спросила Рена. 

 — Это, — ответил за нее Железный Жрец, — это волшебное кольцо, которое можно использовать в качестве талисмана.


**Beam search**

In [ ]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1, 
                        do_sample=False,
                        num_beams=5,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=False,
                        num_beams=5,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
- Темнеет, - заметил эльф. 

 - Да, - согласилась Рена, - но это не значит, что нам не стоит туда соваться. 

 - А почему бы и нет? 

 - Ну, во-

Sample 2
Вдалеке послышался голос. 

 — Что это? — нетерпеливо спросила Рена. 

 — Корабль контрабандистов, — объяснил торговец. — Его захватили пираты. 

 Рена удивленно воззрилась на него.


In [ ]:
# увеличение num_beams улучшения не дает
with torch.no_grad():
    out_1 = model.generate(input_ids_1, 
                        do_sample=False,
                        num_beams=10,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=False,
                        num_beams=10,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
- Темнеет, - заметил эльф. 

 - Да, - согласилась Рена, - но это не значит, что нам больше не о чем с ними говорить. 

 - Да, - согласилась эльфийка, - но это не значит, что

Sample 2
Вдалеке послышался голос. 

 — Что это? — спросила Рена. 

 — Железо, — ответил за нее Меро. — Железо, из которого делают доспехи. 

 — Железо


**Сэмплирование с температурой**

In [ ]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1,
                        do_sample=True,
                        temperature=0.5,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=True,
                        temperature=0.5,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
- Темнеет, - заметил эльф. - А что, если нам попытаться пробраться в лагерь кочевников и незаметно пробраться к ним? 

 - Если мы сумеем это сделать, то сможем и мы, - заметил Рену. - А если

Sample 2
Вдалеке послышался голос.

— Эй, вы там, внизу! — крикнул кто-то.

— Да, — отозвался другой голос. — Это мы, ребята!

На этот раз голоса были более


In [ ]:
# пробуем увеличить температуру => тексты кажутся более разнообразными
with torch.no_grad():
    out_1 = model.generate(input_ids_1,
                        do_sample=True,
                        temperature=1.5,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=True,
                        temperature=1.5,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
- Темнеет, - заметил эльф. Судя по голосу и запаху вина, настроение у ребят из эльфийской дружины было не из радужных… 

 Они молчали и зачарованно смотрели на мерцающие огоньки в кострах. 

 Спустя часа

Sample 2
Вдалеке послышался голос. Крик заставил его насторожиться. «Жрец демонов» снова и снова повторял свое слово. Но ни Дроу, ни Темные маги не обратили на нее внимания. 

 Наконец до них наконец-то доносился новый


In [ ]:
# если добавить параметр top_k, тексты становятся заметно осмысленнее
with torch.no_grad():
    out_1 = model.generate(input_ids_1,
                        do_sample=True,
                        temperature=1.5,
                        top_k=20,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=True,
                        temperature=1.5,
                        top_k=20,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
- Темнеет, - заметил эльф. 

 И действительно, спустя некоторое время над головой Тура появилась луна. 

 «Вот так сюрприз, и они знают, как мы поступаем с нашими пленниками, и готовы разорвать нас голыми руками!»

Sample 2
Вдалеке послышался голос.

В этот момент к ним спешил кто — то из солдат, а за ним и остальные. 

— Роланда нигде нет! Что случилось, черт возьми?! — возмущался один из воинов, пытаясь


In [ ]:
# пробуем увеличить top_k => тексты немного теряют осмысленность
with torch.no_grad():
    out_1 = model.generate(input_ids_1,
                        do_sample=True,
                        temperature=1.5,
                        top_k=30,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=True,
                        temperature=1.5,
                        top_k=30,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
- Темнеет, - заметил эльф. - К ночи мы отсюда уберемся. Мне и этого достаточно. - Мы и так уже успели кое-чего натворить, - отозвалась Кассандра, заметив недовольную гримасу подруги. Та лишь покачала головой.

Sample 2
Вдалеке послышался голос. 

 — А вот это уже лишнее. Вы не собираетесь меня использовать? — насмешливо бросила Джозефина сестре, не сводя взгляда с приближающейся к ним толпы вооруженных всадников. 

 Рена покачала головой. 




### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1) GPT-1 была представлена в статье 'Improving Language Understanding by Generative Pre-Training' (Alec Radford, Karthik Narasimhan, Tim Salimans, Ilya Sutskever, 2018), GPT-2 -- в статье 'Language Models are Unsupervised Multitask Learners' (Alec Radford, Jeffrey Wu, Rewon Child, David Luan, Dario Amodei, Ilya Sutskever, 2019), а GPT-3 -- в статье 'Language Models are Few-Shot Learners' (Tom B. Brown, Benjamin Mann, Nick Ryder, Melanie Subbiah, Jared Kaplan, Prafulla Dhariwal, Arvind Neelakantan, Pranav Shyam, Girish Sastry, Amanda Askell, Sandhini Agarwal , Ariel Herbert-Voss, Gretchen Krueger, Tom Henighan, Rewon Child, Aditya Ramesh, Daniel M. Ziegler, Jeffrey Wu, Clemens Winter, Christopher Hesse, Mark Chen, Eric Sigler, Mateusz Litwin, Scott Gray, Benjamin Chess, Jack Clark, Christopher Berner, Sam McCandlish, Alec Radford, Ilya Sutskever, Dario Amodei, 2020).

2) За основу обучающего корпуса создатели GPT-3 взяли CommonCrawl объемом в триллион слов. Чтобы обеспечить высокое качество текстов в обучающей выборке, создатели GPT-3 сделали следующее: 1) провели автоматическую фильтрацию корпуса CommonCrawl, удалив из него тексты низкого качества (для этой задачи они обучили соответствующий классификатор); 2) провели дедупликацию (с поиском частичных соответствий) внутри документов и между документами, чтобы не допустить избыточности корпуса; 3) добавили известные корпуса высокого качества в обучающую выборку (датасет WebText, интернет-корпуса книг, англоязычную Википедию), чтобы увеличить разнообразие текстов.